In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Classification 
Predict the condition of state - Critical, Moderate or Safe.
Based on total number of cases per 2 weeks.

A new column will be created for this purpose named 'Condition'.
# Regression
Predict the percentage improved of states. 
Based on number of weeks.

In [2]:
# Covid 19 India data
df = pd.read_csv("../Data/covid_19_india.csv")
df.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


In [3]:
# Statewise Testing Details data
df1 = pd.read_csv("../Data/StatewiseTestingDetails.csv")
df1.head

<bound method NDFrame.head of              Date                        State  TotalSamples Negative  \
0      2020-04-17  Andaman and Nicobar Islands        1403.0     1210   
1      2020-04-24  Andaman and Nicobar Islands        2679.0      NaN   
2      2020-04-27  Andaman and Nicobar Islands        2848.0      NaN   
3      2020-05-01  Andaman and Nicobar Islands        3754.0      NaN   
4      2020-05-16  Andaman and Nicobar Islands        6677.0      NaN   
...           ...                          ...           ...      ...   
16331  2021-08-06                  West Bengal    15999961.0      NaN   
16332  2021-08-07                  West Bengal    16045662.0      NaN   
16333  2021-08-08                  West Bengal    16092192.0      NaN   
16334  2021-08-09                  West Bengal    16122345.0      NaN   
16335  2021-08-10                  West Bengal    16162814.0      NaN   

       Positive  
0          12.0  
1          27.0  
2          33.0  
3          33.0  
4  

In [4]:
# Covid vaccine statewise data
df2 = pd.read_csv("../Data/covid_vaccine_statewise.csv")
df2.head()

,Updated On,State,Total Doses Administered,Sessions,Sites,First Dose Administered,Second Dose Administered,Male (Doses Administered),Female (Doses Administered),Transgender (Doses Administered),Covaxin (Doses Administered),CoviShield (Doses Administered),Sputnik V (Doses Administered),AEFI,18-44 Years (Doses Administered),45-60 Years (Doses Administered),60+ Years (Doses Administered),18-44 Years(Individuals Vaccinated),45-60 Years(Individuals Vaccinated),60+ Years(Individuals Vaccinated),Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Individuals Vaccinated
0,16-01-2021,India,48276.0,3455.0,2957.0,48276.0,0.0,NaN,NaN,NaN,579.0,47697.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23757.0,24517.0,2.0,48276.0
1,17-01-2021,India,58604.0,8532.0,4954.0,58604.0,0.0,NaN,NaN,NaN,635.0,57969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27348.0,31252.0,4.0,58604.0
2,18-01-2021,India,99449.0,13611.0,6583.0,99449.0,0.0,NaN,NaN,NaN,1299.0,98150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41361.0,58083.0,5.0,99449.0
3,19-01-2021,India,195525.0,17855.0,7951.0,195525.0,0.0,NaN,NaN,NaN,3017.0,192508.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81901.0,113613.0,11.0,195525.0
4,20-01-2021,India,251280.0,25472.0,10504.0,251280.0,0.0,NaN,NaN,NaN,3946.0,247334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98111.0,153145.0,24.0,251280.0


In [5]:
df.shape

(18110, 9)

In [6]:
# Function to replace names
def replace_names(old,new):
    df["State/UnionTerritory"].replace(old,new,inplace=True)

In [7]:
states = df["State/UnionTerritory"].unique().tolist()
print(states)
print("Total unique states and union territories =", len(states))

['Kerala', 'Telengana', 'Delhi', 'Rajasthan', 'Uttar Pradesh', 'Haryana', 'Ladakh', 'Tamil Nadu', 'Karnataka', 'Maharashtra', 'Punjab', 'Jammu and Kashmir', 'Andhra Pradesh', 'Uttarakhand', 'Odisha', 'Puducherry', 'West Bengal', 'Chhattisgarh', 'Chandigarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh', 'Bihar', 'Manipur', 'Mizoram', 'Andaman and Nicobar Islands', 'Goa', 'Unassigned', 'Assam', 'Jharkhand', 'Arunachal Pradesh', 'Tripura', 'Nagaland', 'Meghalaya', 'Dadra and Nagar Haveli and Daman and Diu', 'Cases being reassigned to states', 'Sikkim', 'Daman & Diu', 'Lakshadweep', 'Telangana', 'Dadra and Nagar Haveli', 'Bihar****', 'Madhya Pradesh***', 'Himanchal Pradesh', 'Karanataka', 'Maharashtra***']
Total unique states and union territories = 46


In [8]:
# Cleaning data 
# Replacing incorrect names of States

replace_names("Maharashtra***", "Maharashtra")
replace_names("Madhya Pradesh***", "Madhya Pradesh")
replace_names("Bihar****", "Bihar")
replace_names("Dadra and Nagar Haveli", "Dadra and Nagar Haveli and Daman and Diu")
replace_names("Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")
replace_names("Himanchal Pradesh", "Himachal Pradesh")
replace_names("Telengana", "Telangana")
replace_names("Karanataka", "Karnataka")


states = df["State/UnionTerritory"].unique().tolist()
print(states)
print("Total unique states and union territories =", len(states))

['Kerala', 'Telangana', 'Delhi', 'Rajasthan', 'Uttar Pradesh', 'Haryana', 'Ladakh', 'Tamil Nadu', 'Karnataka', 'Maharashtra', 'Punjab', 'Jammu and Kashmir', 'Andhra Pradesh', 'Uttarakhand', 'Odisha', 'Puducherry', 'West Bengal', 'Chhattisgarh', 'Chandigarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh', 'Bihar', 'Manipur', 'Mizoram', 'Andaman and Nicobar Islands', 'Goa', 'Unassigned', 'Assam', 'Jharkhand', 'Arunachal Pradesh', 'Tripura', 'Nagaland', 'Meghalaya', 'Dadra and Nagar Haveli and Daman and Diu', 'Cases being reassigned to states', 'Sikkim', 'Lakshadweep']
Total unique states and union territories = 38


In [9]:
# Checking for 'Unassigned' and 'Cases being reassigned to states rows' in the data
print('Cases being reassigned to states =', len(df[df['State/UnionTerritory'] == 'Cases being reassigned to states']))
print('Unassigned =', len(df[df['State/UnionTerritory'] == 'Unassigned']))

Cases being reassigned to states = 60
Unassigned = 3


In [10]:
# Dropping 'Unassigned' and 'Cases being reassigned to states' rows from the data
indexes1 = df[df['State/UnionTerritory'] == 'Cases being reassigned to states'].index
indexes2 = df[df['State/UnionTerritory'] == 'Unassigned'].index
df.drop(indexes1, inplace=True)
df.drop(indexes2, inplace=True)

In [11]:
states = df["State/UnionTerritory"].unique().tolist()
print(states)
print("Total unique states and union territories =", len(states))

['Kerala', 'Telangana', 'Delhi', 'Rajasthan', 'Uttar Pradesh', 'Haryana', 'Ladakh', 'Tamil Nadu', 'Karnataka', 'Maharashtra', 'Punjab', 'Jammu and Kashmir', 'Andhra Pradesh', 'Uttarakhand', 'Odisha', 'Puducherry', 'West Bengal', 'Chhattisgarh', 'Chandigarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh', 'Bihar', 'Manipur', 'Mizoram', 'Andaman and Nicobar Islands', 'Goa', 'Assam', 'Jharkhand', 'Arunachal Pradesh', 'Tripura', 'Nagaland', 'Meghalaya', 'Dadra and Nagar Haveli and Daman and Diu', 'Sikkim', 'Lakshadweep']
Total unique states and union territories = 36


In [12]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Sno,18047.0,NaN,NaN,NaN,9076.41813,5224.916438,1.0,4575.5,9087.0,13598.5,18110.0
Date,18047,560,2021-05-16,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Time,18047,7,8:00 AM,16412,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State/UnionTerritory,18047,36,Kerala,560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ConfirmedIndianNational,18047,62,-,17601,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ConfirmedForeignNational,18047,12,-,17601,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cured,18047.0,NaN,NaN,NaN,279610.209564,615742.451962,0.0,3392.0,33924.0,280679.0,6159676.0
Deaths,18047.0,NaN,NaN,NaN,4066.548734,10935.489348,0.0,34.0,598.0,3669.5,134201.0
Confirmed,18047.0,NaN,NaN,NaN,302063.110822,657060.375285,0.0,4387.0,40269.0,301280.5,6363442.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18047 entries, 0 to 18109
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       18047 non-null  int64 
 1   Date                      18047 non-null  object
 2   Time                      18047 non-null  object
 3   State/UnionTerritory      18047 non-null  object
 4   ConfirmedIndianNational   18047 non-null  object
 5   ConfirmedForeignNational  18047 non-null  object
 6   Cured                     18047 non-null  int64 
 7   Deaths                    18047 non-null  int64 
 8   Confirmed                 18047 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.4+ MB


In [14]:
df.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


In [15]:
# taking the dates and states from the dataframe 
dates = df.iloc[:, 1].values
for i in range(5):
    print(dates[i])

2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03


In [16]:
# splitting the dates and putting them into new lists
months = list()
days = list()
for i in dates:
    months.append(int(i.split('-')[1]))
    days.append(int(i.split('-')[2]))

In [17]:
df.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


In [18]:
# Adding columns days, months to the dataframe
df['days'] = days
df['months'] = months

In [19]:
df.head(42)

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,days,months
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1,30,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1,31,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2,1,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3,2,2
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3,3,2
5,6,2020-02-04,6:00 PM,Kerala,3,0,0,0,3,4,2
6,7,2020-02-05,6:00 PM,Kerala,3,0,0,0,3,5,2
7,8,2020-02-06,6:00 PM,Kerala,3,0,0,0,3,6,2
8,9,2020-02-07,6:00 PM,Kerala,3,0,0,0,3,7,2
9,10,2020-02-08,6:00 PM,Kerala,3,0,0,0,3,8,2


In [20]:
df.shape

(18047, 11)

In [21]:
# creating condition column by checking the 
# confirmed cases based on past 2 weeks
break_point = 0
condition = []
graph = []
day_wise = {}
for state in states:
    day_wise[state] = states.count(state) - 1
day_wise

{'Kerala': 0,
 'Telangana': 0,
 'Delhi': 0,
 'Rajasthan': 0,
 'Uttar Pradesh': 0,
 'Haryana': 0,
 'Ladakh': 0,
 'Tamil Nadu': 0,
 'Karnataka': 0,
 'Maharashtra': 0,
 'Punjab': 0,
 'Jammu and Kashmir': 0,
 'Andhra Pradesh': 0,
 'Uttarakhand': 0,
 'Odisha': 0,
 'Puducherry': 0,
 'West Bengal': 0,
 'Chhattisgarh': 0,
 'Chandigarh': 0,
 'Gujarat': 0,
 'Himachal Pradesh': 0,
 'Madhya Pradesh': 0,
 'Bihar': 0,
 'Manipur': 0,
 'Mizoram': 0,
 'Andaman and Nicobar Islands': 0,
 'Goa': 0,
 'Assam': 0,
 'Jharkhand': 0,
 'Arunachal Pradesh': 0,
 'Tripura': 0,
 'Nagaland': 0,
 'Meghalaya': 0,
 'Dadra and Nagar Haveli and Daman and Diu': 0,
 'Sikkim': 0,
 'Lakshadweep': 0}

In [22]:
day_wise_confirmed = []
day_wise_confirmed.append(day_wise.copy())
dates = df.Date.unique()
dates

array(['2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02',
       '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
       '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10',
       '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
       '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18',
       '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22',
       '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26',
       '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01',
       '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
       '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
       '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17',
       '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21',
       '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',
       '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
       '2020-03-30', '2020-03-31', '2020-04-01', '2020-

In [23]:
# creating dictionary and nesting it with list to store the sum of confirmed cases in particular break point
c = 0
for date in dates:
    for state,positive in day_wise.items():
        current_day = df[(df["State/UnionTerritory"]==state) & (df["Date"]==date)].Confirmed.sum()
        day_wise[state] = current_day
    print(day_wise)
    day_wise_confirmed.append(day_wise.copy())

{'Kerala': 1, 'Telangana': 0, 'Delhi': 0, 'Rajasthan': 0, 'Uttar Pradesh': 0, 'Haryana': 0, 'Ladakh': 0, 'Tamil Nadu': 0, 'Karnataka': 0, 'Maharashtra': 0, 'Punjab': 0, 'Jammu and Kashmir': 0, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Pradesh': 0, 'Bihar': 0, 'Manipur': 0, 'Mizoram': 0, 'Andaman and Nicobar Islands': 0, 'Goa': 0, 'Assam': 0, 'Jharkhand': 0, 'Arunachal Pradesh': 0, 'Tripura': 0, 'Nagaland': 0, 'Meghalaya': 0, 'Dadra and Nagar Haveli and Daman and Diu': 0, 'Sikkim': 0, 'Lakshadweep': 0}
{'Kerala': 1, 'Telangana': 0, 'Delhi': 0, 'Rajasthan': 0, 'Uttar Pradesh': 0, 'Haryana': 0, 'Ladakh': 0, 'Tamil Nadu': 0, 'Karnataka': 0, 'Maharashtra': 0, 'Punjab': 0, 'Jammu and Kashmir': 0, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Prades

{'Kerala': 3, 'Telangana': 0, 'Delhi': 0, 'Rajasthan': 0, 'Uttar Pradesh': 0, 'Haryana': 0, 'Ladakh': 0, 'Tamil Nadu': 0, 'Karnataka': 0, 'Maharashtra': 0, 'Punjab': 0, 'Jammu and Kashmir': 0, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Pradesh': 0, 'Bihar': 0, 'Manipur': 0, 'Mizoram': 0, 'Andaman and Nicobar Islands': 0, 'Goa': 0, 'Assam': 0, 'Jharkhand': 0, 'Arunachal Pradesh': 0, 'Tripura': 0, 'Nagaland': 0, 'Meghalaya': 0, 'Dadra and Nagar Haveli and Daman and Diu': 0, 'Sikkim': 0, 'Lakshadweep': 0}
{'Kerala': 3, 'Telangana': 0, 'Delhi': 0, 'Rajasthan': 0, 'Uttar Pradesh': 0, 'Haryana': 0, 'Ladakh': 0, 'Tamil Nadu': 0, 'Karnataka': 0, 'Maharashtra': 0, 'Punjab': 0, 'Jammu and Kashmir': 0, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Prades

{'Kerala': 3, 'Telangana': 0, 'Delhi': 0, 'Rajasthan': 0, 'Uttar Pradesh': 0, 'Haryana': 0, 'Ladakh': 0, 'Tamil Nadu': 0, 'Karnataka': 0, 'Maharashtra': 0, 'Punjab': 0, 'Jammu and Kashmir': 0, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Pradesh': 0, 'Bihar': 0, 'Manipur': 0, 'Mizoram': 0, 'Andaman and Nicobar Islands': 0, 'Goa': 0, 'Assam': 0, 'Jharkhand': 0, 'Arunachal Pradesh': 0, 'Tripura': 0, 'Nagaland': 0, 'Meghalaya': 0, 'Dadra and Nagar Haveli and Daman and Diu': 0, 'Sikkim': 0, 'Lakshadweep': 0}
{'Kerala': 3, 'Telangana': 0, 'Delhi': 0, 'Rajasthan': 0, 'Uttar Pradesh': 0, 'Haryana': 0, 'Ladakh': 0, 'Tamil Nadu': 0, 'Karnataka': 0, 'Maharashtra': 0, 'Punjab': 0, 'Jammu and Kashmir': 0, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Prades

{'Kerala': 9, 'Telangana': 1, 'Delhi': 4, 'Rajasthan': 15, 'Uttar Pradesh': 7, 'Haryana': 2, 'Ladakh': 2, 'Tamil Nadu': 1, 'Karnataka': 1, 'Maharashtra': 2, 'Punjab': 1, 'Jammu and Kashmir': 1, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Pradesh': 0, 'Bihar': 0, 'Manipur': 0, 'Mizoram': 0, 'Andaman and Nicobar Islands': 0, 'Goa': 0, 'Assam': 0, 'Jharkhand': 0, 'Arunachal Pradesh': 0, 'Tripura': 0, 'Nagaland': 0, 'Meghalaya': 0, 'Dadra and Nagar Haveli and Daman and Diu': 0, 'Sikkim': 0, 'Lakshadweep': 0}
{'Kerala': 15, 'Telangana': 1, 'Delhi': 4, 'Rajasthan': 15, 'Uttar Pradesh': 7, 'Haryana': 2, 'Ladakh': 2, 'Tamil Nadu': 1, 'Karnataka': 4, 'Maharashtra': 5, 'Punjab': 1, 'Jammu and Kashmir': 1, 'Andhra Pradesh': 0, 'Uttarakhand': 0, 'Odisha': 0, 'Puducherry': 0, 'West Bengal': 0, 'Chhattisgarh': 0, 'Chandigarh': 0, 'Gujarat': 0, 'Himachal Pradesh': 0, 'Madhya Pra

{'Kerala': 67, 'Telangana': 32, 'Delhi': 29, 'Rajasthan': 28, 'Uttar Pradesh': 31, 'Haryana': 26, 'Ladakh': 13, 'Tamil Nadu': 9, 'Karnataka': 33, 'Maharashtra': 74, 'Punjab': 21, 'Jammu and Kashmir': 4, 'Andhra Pradesh': 7, 'Uttarakhand': 3, 'Odisha': 2, 'Puducherry': 1, 'West Bengal': 7, 'Chhattisgarh': 1, 'Chandigarh': 6, 'Gujarat': 29, 'Himachal Pradesh': 2, 'Madhya Pradesh': 6, 'Bihar': 2, 'Manipur': 0, 'Mizoram': 0, 'Andaman and Nicobar Islands': 0, 'Goa': 0, 'Assam': 0, 'Jharkhand': 0, 'Arunachal Pradesh': 0, 'Tripura': 0, 'Nagaland': 0, 'Meghalaya': 0, 'Dadra and Nagar Haveli and Daman and Diu': 0, 'Sikkim': 0, 'Lakshadweep': 0}
{'Kerala': 95, 'Telangana': 35, 'Delhi': 30, 'Rajasthan': 32, 'Uttar Pradesh': 33, 'Haryana': 28, 'Ladakh': 13, 'Tamil Nadu': 15, 'Karnataka': 37, 'Maharashtra': 89, 'Punjab': 29, 'Jammu and Kashmir': 4, 'Andhra Pradesh': 8, 'Uttarakhand': 4, 'Odisha': 2, 'Puducherry': 1, 'West Bengal': 9, 'Chhattisgarh': 1, 'Chandigarh': 7, 'Gujarat': 33, 'Himachal Prad

...

{'Kerala': 2735958, 'Telangana': 604880, 'Delhi': 1431270, 'Rajasthan': 949961, 'Uttar Pradesh': 1702937, 'Haryana': 766129, 'Ladakh': 19611, 'Tamil Nadu': 2366493, 'Karnataka': 2771969, 'Maharashtra': 5917121, 'Punjab': 588525, 'Jammu and Kashmir': 308011, 'Andhra Pradesh': 1814393, 'Uttarakhand': 337175, 'Odisha': 856121, 'Puducherry': 112837, 'West Bengal': 1464776, 'Chhattisgarh': 987563, 'Chandigarh': 61160, 'Gujarat': 820726, 'Himachal Pradesh': 198876, 'Madhya Pradesh': 788425, 'Bihar': 717539, 'Manipur': 60311, 'Mizoram': 15631, 'Andaman and Nicobar Islands': 7269, 'Goa': 162721, 'Assam': 463175, 'Jharkhand': 343609, 'Arunachal Pradesh': 31648, 'Tripura': 59849, 'Nagaland': 23753, 'Meghalaya': 42309, 'Dadra and Nagar Haveli and Daman and Diu': 10464, 'Sikkim': 18450, 'Lakshadweep': 9236}
{'Kerala': 2748204, 'Telangana': 606436, 'Delhi': 1431498, 'Rajasthan': 950133, 'Uttar Pradesh': 1703207, 'Haryana': 766357, 'Ladakh': 19649, 'Tamil Nadu': 2378298, 'Karnataka': 2777010, 'Mahar

{'Kerala': 2877989, 'Telangana': 619865, 'Delhi': 1433675, 'Rajasthan': 951967, 'Uttar Pradesh': 1705384, 'Haryana': 768263, 'Ladakh': 19941, 'Tamil Nadu': 2460747, 'Karnataka': 2831026, 'Maharashtra': 6026847, 'Punjab': 594586, 'Jammu and Kashmir': 314316, 'Andhra Pradesh': 1875622, 'Uttarakhand': 339537, 'Odisha': 897062, 'Puducherry': 116414, 'West Bengal': 1493113, 'Chhattisgarh': 993045, 'Chandigarh': 61604, 'Gujarat': 823132, 'Himachal Pradesh': 201547, 'Madhya Pradesh': 789657, 'Bihar': 721114, 'Manipur': 67317, 'Mizoram': 19324, 'Andaman and Nicobar Islands': 7446, 'Goa': 165883, 'Assam': 499121, 'Jharkhand': 345259, 'Arunachal Pradesh': 34767, 'Tripura': 64644, 'Nagaland': 24850, 'Meghalaya': 47985, 'Dadra and Nagar Haveli and Daman and Diu': 10534, 'Sikkim': 19941, 'Lakshadweep': 9659}
{'Kerala': 2888894, 'Telangana': 620613, 'Delhi': 1433934, 'Rajasthan': 952129, 'Uttar Pradesh': 1705596, 'Haryana': 768378, 'Ladakh': 19964, 'Tamil Nadu': 2465874, 'Karnataka': 2834630, 'Mahar

{'Kerala': 3025466, 'Telangana': 629785, 'Delhi': 1434873, 'Rajasthan': 952956, 'Uttar Pradesh': 1707044, 'Haryana': 769148, 'Ladakh': 20155, 'Tamil Nadu': 2510059, 'Karnataka': 2864868, 'Maharashtra': 6131976, 'Punjab': 597195, 'Jammu and Kashmir': 318023, 'Andhra Pradesh': 1914213, 'Uttarakhand': 341023, 'Odisha': 932330, 'Puducherry': 118571, 'West Bengal': 1509218, 'Chhattisgarh': 997035, 'Chandigarh': 61794, 'Gujarat': 824091, 'Himachal Pradesh': 203245, 'Madhya Pradesh': 790096, 'Bihar': 722965, 'Manipur': 75180, 'Mizoram': 23014, 'Andaman and Nicobar Islands': 7492, 'Goa': 168210, 'Assam': 527200, 'Jharkhand': 346168, 'Arunachal Pradesh': 38682, 'Tripura': 69550, 'Nagaland': 25727, 'Meghalaya': 53244, 'Dadra and Nagar Haveli and Daman and Diu': 10579, 'Sikkim': 21795, 'Lakshadweep': 9980}
{'Kerala': 3039029, 'Telangana': 630514, 'Delhi': 1434954, 'Rajasthan': 953018, 'Uttar Pradesh': 1707127, 'Haryana': 769205, 'Ladakh': 20167, 'Tamil Nadu': 2513098, 'Karnataka': 2867158, 'Mahar

{'Kerala': 3187716, 'Telangana': 638030, 'Delhi': 1435609, 'Rajasthan': 953415, 'Uttar Pradesh': 1707953, 'Haryana': 769605, 'Ladakh': 20278, 'Tamil Nadu': 2539277, 'Karnataka': 2886702, 'Maharashtra': 6229596, 'Punjab': 598455, 'Jammu and Kashmir': 320112, 'Andhra Pradesh': 1944222, 'Uttarakhand': 341536, 'Odisha': 958059, 'Puducherry': 119823, 'West Bengal': 1519599, 'Chhattisgarh': 1000358, 'Chandigarh': 61907, 'Gujarat': 824546, 'Himachal Pradesh': 204618, 'Madhya Pradesh': 791689, 'Bihar': 724081, 'Manipur': 86723, 'Mizoram': 29020, 'Andaman and Nicobar Islands': 7518, 'Goa': 169971, 'Assam': 550878, 'Jharkhand': 346778, 'Arunachal Pradesh': 43804, 'Tripura': 74846, 'Nagaland': 26838, 'Meghalaya': 58275, 'Dadra and Nagar Haveli and Daman and Diu': 10612, 'Sikkim': 23796, 'Lakshadweep': 10092}
{'Kerala': 3205197, 'Telangana': 638721, 'Delhi': 1435671, 'Rajasthan': 953437, 'Uttar Pradesh': 1708005, 'Haryana': 769640, 'Ladakh': 20282, 'Tamil Nadu': 2541168, 'Karnataka': 2888341, 'Mah

{'Kerala': 3411489, 'Telangana': 645406, 'Delhi': 1436350, 'Rajasthan': 953688, 'Uttar Pradesh': 1708476, 'Haryana': 769942, 'Ladakh': 20340, 'Tamil Nadu': 2561587, 'Karnataka': 2906999, 'Maharashtra': 6310194, 'Punjab': 599130, 'Jammu and Kashmir': 321607, 'Andhra Pradesh': 1968462, 'Uttarakhand': 342161, 'Odisha': 978705, 'Puducherry': 121005, 'West Bengal': 1528720, 'Chhattisgarh': 1002222, 'Chandigarh': 61954, 'Gujarat': 824900, 'Himachal Pradesh': 206161, 'Madhya Pradesh': 791845, 'Bihar': 724880, 'Manipur': 99331, 'Mizoram': 39363, 'Andaman and Nicobar Islands': 7539, 'Goa': 171205, 'Assam': 566982, 'Jharkhand': 347200, 'Arunachal Pradesh': 48260, 'Tripura': 78722, 'Nagaland': 27945, 'Meghalaya': 65589, 'Dadra and Nagar Haveli and Daman and Diu': 10650, 'Sikkim': 26754, 'Lakshadweep': 10195}
{'Kerala': 3425473, 'Telangana': 645997, 'Delhi': 1436401, 'Rajasthan': 953704, 'Uttar Pradesh': 1708500, 'Haryana': 769956, 'Ladakh': 20345, 'Tamil Nadu': 2563544, 'Karnataka': 2908284, 'Mah

In [34]:
day_wise_confirmed.pop(0)

[{'Kerala': 1,
  'Telangana': 0,
  'Delhi': 0,
  'Rajasthan': 0,
  'Uttar Pradesh': 0,
  'Haryana': 0,
  'Ladakh': 0,
  'Tamil Nadu': 0,
  'Karnataka': 0,
  'Maharashtra': 0,
  'Punjab': 0,
  'Jammu and Kashmir': 0,
  'Andhra Pradesh': 0,
  'Uttarakhand': 0,
  'Odisha': 0,
  'Puducherry': 0,
  'West Bengal': 0,
  'Chhattisgarh': 0,
  'Chandigarh': 0,
  'Gujarat': 0,
  'Himachal Pradesh': 0,
  'Madhya Pradesh': 0,
  'Bihar': 0,
  'Manipur': 0,
  'Mizoram': 0,
  'Andaman and Nicobar Islands': 0,
  'Goa': 0,
  'Assam': 0,
  'Jharkhand': 0,
  'Arunachal Pradesh': 0,
  'Tripura': 0,
  'Nagaland': 0,
  'Meghalaya': 0,
  'Dadra and Nagar Haveli and Daman and Diu': 0,
  'Sikkim': 0,
  'Lakshadweep': 0},
 {'Kerala': 1,
  'Telangana': 0,
  'Delhi': 0,
  'Rajasthan': 0,
  'Uttar Pradesh': 0,
  'Haryana': 0,
  'Ladakh': 0,
  'Tamil Nadu': 0,
  'Karnataka': 0,
  'Maharashtra': 0,
  'Punjab': 0,
  'Jammu and Kashmir': 0,
  'Andhra Pradesh': 0,
  'Uttarakhand': 0,
  'Odisha': 0,
  'Puducherry': 0,
  

In [25]:
df.tail()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,days,months
18105,18106,2021-08-11,8:00 AM,Telangana,-,-,638410,3831,650353,11,8
18106,18107,2021-08-11,8:00 AM,Tripura,-,-,77811,773,80660,11,8
18107,18108,2021-08-11,8:00 AM,Uttarakhand,-,-,334650,7368,342462,11,8
18108,18109,2021-08-11,8:00 AM,Uttar Pradesh,-,-,1685492,22775,1708812,11,8
18109,18110,2021-08-11,8:00 AM,West Bengal,-,-,1506532,18252,1534999,11,8


In [38]:
week_wise = {}
for state in states:
    week_wise[state] = states.count(state) - 1
week_wise
week_wise_confirmed = []

In [39]:
count = 0
for day in day_wise_confirmed:
    for state,confirmed in day.items():
        week_wise[state] += confirmed
    if count==13:
        week_wise_confirmed.append(week_wise.copy())
        for state in states:
            week_wise[state] = states.count(state) - 1
        count = 0
        continue
    count +=1

In [40]:
week_wise_confirmed

[{'Kerala': 37,
  'Telangana': 0,
  'Delhi': 0,
  'Rajasthan': 0,
  'Uttar Pradesh': 0,
  'Haryana': 0,
  'Ladakh': 0,
  'Tamil Nadu': 0,
  'Karnataka': 0,
  'Maharashtra': 0,
  'Punjab': 0,
  'Jammu and Kashmir': 0,
  'Andhra Pradesh': 0,
  'Uttarakhand': 0,
  'Odisha': 0,
  'Puducherry': 0,
  'West Bengal': 0,
  'Chhattisgarh': 0,
  'Chandigarh': 0,
  'Gujarat': 0,
  'Himachal Pradesh': 0,
  'Madhya Pradesh': 0,
  'Bihar': 0,
  'Manipur': 0,
  'Mizoram': 0,
  'Andaman and Nicobar Islands': 0,
  'Goa': 0,
  'Assam': 0,
  'Jharkhand': 0,
  'Arunachal Pradesh': 0,
  'Tripura': 0,
  'Nagaland': 0,
  'Meghalaya': 0,
  'Dadra and Nagar Haveli and Daman and Diu': 0,
  'Sikkim': 0,
  'Lakshadweep': 0},
 {'Kerala': 42,
  'Telangana': 0,
  'Delhi': 0,
  'Rajasthan': 0,
  'Uttar Pradesh': 0,
  'Haryana': 0,
  'Ladakh': 0,
  'Tamil Nadu': 0,
  'Karnataka': 0,
  'Maharashtra': 0,
  'Punjab': 0,
  'Jammu and Kashmir': 0,
  'Andhra Pradesh': 0,
  'Uttarakhand': 0,
  'Odisha': 0,
  'Puducherry': 0,


In [27]:
# creating condition column by checking the 
# confirmed cases based on past 2 weeks
# number_of_cases = 0
# count = 0
# confirmedCases = df.Confirmed.values
# states = df['State/UnionTerritory'].values
# confirmed = 0
# condition = list()
# j = 0
# stateCondition = [0] * 36
# lastWeekCases = -1
 
# for i in range(18047):
#     if count == 14:
#         count = 0
#     # checking states/union territories so that 
#     # the values are unique
#     # if states[j] == df['State/UnionTerritory'].values and stateCondition[j] == 0: 
#     for i in states:
#         number_of_cases += confirmedCases[i]
#         if number_of_cases < lastWeekCases and number_of_cases < 100:
#             condition.append('Safe')
#         elif number_of_cases == lastWeekCases and number_of_cases > 100 and number_of_cases < 10000:
#             condition.append('Moderate')
#         elif number_of_cases == (lastWeekCases + 10) or number_of_cases == (lastWeekCases - 10):
#             condition.append('Moderate')
#         elif number_of_cases > lastWeekCases and number_of_cases > 10000:
#             condition.append('Critical')
#     count += 1
#     print(condition)